# 도로명주소 위경도 좌표변환
- `geopy` 패키지 이용한 지오코딩

In [ ]:
# For Google Colab
from google.colab import drive
drive.mount('/content/Mydrive')
%cd "/content/Mydrive/MyDrive/IOT_hackathon/Github/IOT_hackathon"

Drive already mounted at /content/Mydrive; to attempt to forcibly remount, call drive.mount("/content/Mydrive", force_remount=True).
/content/Mydrive/MyDrive/IOT_hackathon/Github/IOT_hackathon


In [ ]:
import pandas as pd
from geopy.geocoders import Nominatim
import numpy as np
from tqdm import tqdm

geo_local = Nominatim(user_agent='South Korea', timeout=None)
df = pd.read_csv('data/seoul_schools.csv', encoding='cp949')
df = df[['학교종류명','도로명주소']]

In [ ]:
df.학교종류명.unique()

NameError: ignored

In [ ]:
# 학교 종류가 많으므로 전처리
df.loc[df.학교종류명 == '각종학교(초)','학교종류명'] = '초등학교'
df.loc[df.학교종류명 == '각종학교(중)','학교종류명'] = '중학교'
df.loc[df.학교종류명 == '각종학교(고)','학교종류명'] = '고등학교'
df.loc[df.학교종류명 == '평생학교(초)-3년6학기','학교종류명'] = '초등학교'
df.loc[df.학교종류명 == '평생학교(중)-2년6학기','학교종류명'] = '중학교'
df.loc[df.학교종류명 == '평생학교(중)-2년6학기','학교종류명'] = '중학교'
df.loc[df.학교종류명 == '평생학교(고)-2년6학기','학교종류명'] = '고등학교'
df.loc[df.학교종류명 == '방송통신고등학교','학교종류명'] = '고등학교'

In [ ]:
# 지오코딩 (주소를 통해 위도, 경도 좌표 얻기)

def geocoding(address):
    geo = geo_local.geocode(address)

    return geo.latitude, geo.longitude

In [ ]:
lat, long = [], []
for i in tqdm(range(len(df))):
    try: 
        lat_i, long_i = geocoding(df.도로명주소[i])

    except AttributeError:
        lat_i, long_i = 0.0, 0.0
    lat.append(lat_i)
    long.append(long_i)

df['lat'] = lat
df['long'] = long

100%|██████████| 3852/3852 [32:04<00:00,  2.00it/s]


In [ ]:
df.loc[df.lat == 0.0 , :]

,학교종류명,도로명주소,lat,long
77,중학교,서울특별시 동작구 사당로2길 2-19,0.0,0.0
97,중학교,서울특별시 동작구 상도로13라길 22,0.0,0.0
143,초등학교,서울특별시 송파구 위례북로2길 12,0.0,0.0
144,중학교,서울특별시 강동구 아리수로93다길 1,0.0,0.0
145,초등학교,서울특별시 강동구 아리수로93다길 1,0.0,0.0
...,...,...,...,...
3673,고등학교,서울특별시 서초구 반포대로27길 29,0.0,0.0
3674,고등학교,서울특별시 서초구 반포대로27길 29,0.0,0.0
3675,고등학교,서울특별시 서초구 반포대로27길 29,0.0,0.0
3676,고등학교,서울특별시 서초구 반포대로27길 29,0.0,0.0


In [ ]:
df.to_csv('./data/seoul_schools_location.csv', index=False) # File Saving

In [ ]:
## 대학정보
univ = pd.read_csv('data/Univ_College_Loc(2022).csv', encoding='cp949',)

In [ ]:
lat, long = [], []
for i in tqdm(range(len(univ))):
    try: 
        lat_i, long_i = geocoding(univ.주소[i])

    except AttributeError:
        lat_i, long_i = 0.0, 0.0
    lat.append(lat_i)
    long.append(long_i)

univ['lat'] = lat
univ['long'] = long

100%|██████████| 64/64 [00:32<00:00,  2.00it/s]


In [ ]:
univ.to_csv('data/seoul_univ_location.csv', index=False)

In [ ]:
# 자전거보관소 
bicycle_storage = pd.read_csv('data/bicycle_storage(2020-2022).csv', encoding='cp949', usecols=['WGS84위도','WGS84경도','데이터기준일자'])

In [ ]:
bicycle_storage.columns = ['lat','long','date']
bicycle_storage.date = pd.to_datetime(bicycle_storage.date)
bicycle_storage = bicycle_storage.loc[bicycle_storage.date < pd.Timestamp(2022,1,1),:]
bicycle_storage.to_csv('bicycle_storage.csv')

In [ ]:
# 식당
restaurants = pd.read_csv('data/restaurant_seoul(2021-2022).csv', encoding='cp949')
restaurants

,번호,소재지면적,도로명전체주소,사업장명,최종수정시점,데이터갱신일자,업태구분명,좌표정보(X),좌표정보(Y),위생업태명
0,1,26.40,"서울특별시 강동구 천중로 237, 1층 (길동)",플랫,20220823155406,2022-08-25 00:22:37.0,기타,212899.525378,448741.141661,기타
1,2,NaN,"서울특별시 강동구 천호대로 1005, 현대백화점 지2층 (천호동)",카르네(이탈리) 한시적영업,20220823172803,2022-08-25 00:22:37.0,기타,210929.919694,448537.406728,기타
2,3,38.25,"서울특별시 송파구 양재대로72길 17, 현대백조아파트 상가1동 1층 101-3호 (...",동양,20220823145030,2022-08-25 00:22:37.0,일식,211358.758041,445248.724359,일식
3,4,19.00,"서울특별시 송파구 송이로14길 13, 국토빌딩 1층 (가락동)",계절간식,20220823113436,2022-08-25 00:22:37.0,기타,210375.063574,444053.720259,기타
4,5,29.00,"서울특별시 송파구 송이로14길 13, 국토빌딩 1층 (가락동)",ROSSI 카페,20220823112946,2022-08-25 00:22:37.0,기타,210375.063574,444053.720259,기타
...,...,...,...,...,...,...,...,...,...,...
11873,11633,35.64,"서울특별시 강동구 상일로15길 22, 1층 (고덕동)",비비큐 고덕센트럴점,20210830142010,2021-09-01 00:22:50.0,호프/통닭,214946.249742,450680.438962,호프/통닭
11874,11634,73.00,"서울특별시 강동구 천호대로176길 9, 1층 101,102호 (성내동)",예찬정도시락,20210830105700,2021-09-01 00:22:50.0,김밥(도시락),211986.355623,447973.167204,김밥(도시락)
11875,10995,223.71,"서울특별시 송파구 송파대로 111, 202동 지하1층 B127,B129,B131,B...",대찬낙지,20210902152242,2021-09-04 02:40:00.0,한식,210868.397497,442053.227933,한식
11876,11639,33.00,"서울특별시 강동구 천중로 216, 1층 (길동)",파전나라,20220127150838,2022-01-29 02:40:00.0,한식,212680.903683,448720.810378,한식
